<a href="https://colab.research.google.com/github/vlad-danaila/ml-cancer-detection/blob/master/Cancer_Detection_Ensable_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Download dependencies**

In [0]:
!pip3 install sklearn matplotlib GPUtil

  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=9551351960dc3403d4cc8647d6ef1fe058d2be5a7d77d5f013c3bbde5a10c330
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil


In [0]:
!pip3 install torch==1.3.1+cu92 torchvision==0.4.2+cu92 -f https://download.pytorch.org/whl/torch_stable.html 

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 621.4MB 72kB/s s eta 0:00:01     |███████████████▎                | 296.4MB 85.9MB/s eta 0:00:04     |███████████████▉                | 306.9MB 85.9MB/s eta 0:00:04     |███████████████████▋            | 380.5MB 52.0MB/s eta 0:00:05     |████████████████████▊           | 403.0MB 52.0MB/s eta 0:00:05     |████████████████████████▋       | 478.5MB 2.7MB/s eta 0:00:53     |██████████████████████████      | 504.5MB 2.7MB/s eta 0:00:44
     |████████████████████████████████| 10.1MB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 12.6MB/s eta 0:00:01


**Download Data**

Mount my google drive, where I stored the dataset.

In [0]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception as e:
    print(e)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In order to acquire the dataset please navigate to:

https://ieee-dataport.org/documents/cervigram-image-dataset

Unzip the dataset into the folder "dataset".

For your environment, please adjust the paths accordingly.

In [0]:
!rm -vrf "dataset"
!mkdir "dataset"
!cp -r "/content/drive/My Drive/Studiu doctorat leziuni cervicale/cervigram-image-dataset-v2.zip" "dataset/cervigram-image-dataset-v2.zip"
# !cp -r "cervigram-image-dataset-v2.zip" "dataset/cervigram-image-dataset-v2.zip"
!unzip "dataset/cervigram-image-dataset-v2.zip" -d "dataset"

removed 'dataset/data/train/3/20160406014/20160406155835.jpg'
removed 'dataset/data/train/3/20160406014/20160406160345.jpg'
removed 'dataset/data/train/3/20160406014/20160406160153.jpg'
removed 'dataset/data/train/3/20160406014/20160406160152.jpg'
removed 'dataset/data/train/3/20160406014/20160406160125.jpg'
removed 'dataset/data/train/3/20160406014/20160406160044.jpg'
removed 'dataset/data/train/3/20160406014/20160406160059.jpg'
removed directory 'dataset/data/train/3/20160406014'
removed 'dataset/data/train/3/20150930010/20150930160649.jpg'
removed 'dataset/data/train/3/20150930010/20150930161006.jpg'
removed 'dataset/data/train/3/20150930010/20150930160926.jpg'
removed 'dataset/data/train/3/20150930010/20150930161002.jpg'
removed 'dataset/data/train/3/20150930010/20150930161104.jpg'
removed 'dataset/data/train/3/20150930010/20150930160831.jpg'
removed 'dataset/data/train/3/20150930010/20150930160900.jpg'
removed directory 'dataset/data/train/3/20150930010'
removed 'dataset/data/trai

**Constants**

For your environment, please modify the paths accordingly.  

In [0]:
TRAIN_PATH = '/content/dataset/data/train/'
TEST_PATH = '/content/dataset/data/test/'
# TRAIN_PATH = 'dataset/data/train/'
# TEST_PATH = 'dataset/data/test/'

CROP_SIZE = 260
IMAGE_SIZE = 224
BATCH_SIZE = 100

prefix = '/content/drive/My Drive/Studiu doctorat leziuni cervicale/V2/Chekpoints & Notebooks/'
CHECKPOINT_NATURAL_IMG_MODEL = prefix + 'Cancer Detection MobileNetV2 All Natural Images Full Conv32-0.7 6 Dec.tar'
CHECKPOINT_GREEN_LENS_IMG_MODEL = prefix + 'Cancer_Detection_MobileNetV2_Green_Lens_2_Dec Full Conv 64 0.7.tar'
CHECKPOINT_IODINE_SOLUTION_IMG_MODEL = prefix + 'Cancer_Detection_MobileNetV2_Iodine_1_Dec Full Conv32.tar'
CHECKPOINT_ENSAMBLE = prefix + 'Cancer Detection - Ensable Conv 7 Dec.tar'

**Imports**

In [0]:
import torch as t
import torchvision as tv
import numpy as np
import PIL as pil
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.nn import Linear, BCEWithLogitsLoss
import sklearn as sk
import sklearn.metrics
from os import listdir
import time
import random

**Deterministic Measurements**

This statements help making the experiments reproducible by fixing the random seeds. Despite fixing the random seeds, experiments are usually not reproducible using different PyTorch releases, commits, platforms or between CPU and GPU executions. Please find more details in the PyTorch documentation:

https://pytorch.org/docs/stable/notes/randomness.html

In [0]:
SEED = 0
t.manual_seed(SEED)
t.cuda.manual_seed(SEED)
t.cuda.manual_seed_all(SEED)
t.backends.cudnn.deterministic = True
t.backends.cudnn.benchmark = False
np.random.seed(SEED)
random.seed(SEED)

**Memory Stats**

In [0]:
import GPUtil
def memory_stats():
  for gpu in GPUtil.getGPUs():
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
memory_stats()

GPU RAM Free: 13985MB | Used: 2295MB | Util  14% | Total 16280MB


**Loading Data**

The dataset is structured in multiple small folders of 7 images each. This generator iterates through the folders and returns the category and 7 paths: one for each image in the folder. The paths are ordered; the order is important since each folder contains 3 types of images, first 5 are with acetic acid solution and the last two are through a green lens and having iodine solution(a solution of a dark red color).

In [0]:
def sortByLastDigits(elem):
  chars = [c for c in elem if c.isdigit()]
  return 0 if len(chars) == 0 else int(''.join(chars))

def getImagesPaths(root_path):
  for  class_folder in [root_path + f for f in listdir(root_path)]:
      category = int(class_folder[-1])
      for case_folder in listdir(class_folder):
        case_folder_path = class_folder + '/' + case_folder + '/'
        img_files = [case_folder_path + file_name for file_name in listdir(case_folder_path)]
        yield category, sorted(img_files, key = sortByLastDigits)

We define 4 datasets, which load: natural images, images taken through a green lens, images where the doctor applied iodine solution (which gives a dark red color) and all images. Each dataset has dynamic and static transformations which could be applied to the data. The static transformations are applied on the initialization of the dataset, while the dynamic ones are applied when loading each batch of data.

In [0]:
class SimpleImagesDataset(t.utils.data.Dataset):
  def __init__(self, root_path, transforms_x_static = None, transforms_x_dynamic = None, transforms_y_static = None, transforms_y_dynamic = None):
    self.dataset = []
    self.transforms_x = transforms_x_dynamic
    self.transforms_y = transforms_y_dynamic
    for category, img_files in getImagesPaths(root_path):
      for i in range(5):
        img = pil.Image.open(img_files[i])
        if transforms_x_static != None:
          img = transforms_x_static(img)
        if transforms_y_static != None:
          category = transforms_y_static(category)
        self.dataset.append((img, category))    
  
  def __getitem__(self, i):
    x, y = self.dataset[i]
    if self.transforms_x != None:
      x = self.transforms_x(x)
    if self.transforms_y != None:
      y = self.transforms_y(y)
    return x, y

  def __len__(self):
    return len(self.dataset)

class GreenLensImagesDataset(SimpleImagesDataset):
  def __init__(self, root_path, transforms_x_static = None, transforms_x_dynamic = None, transforms_y_static = None, transforms_y_dynamic = None):
    self.dataset = []
    self.transforms_x = transforms_x_dynamic
    self.transforms_y = transforms_y_dynamic
    for category, img_files in getImagesPaths(root_path):
        # Only the green lens image
        img = pil.Image.open(img_files[-2]) 
        if transforms_x_static != None:
          img = transforms_x_static(img)
        if transforms_y_static != None:
          category = transforms_y_static(category)
        self.dataset.append((img, category))    

class RedImagesDataset(SimpleImagesDataset):
  def __init__(self, root_path, transforms_x_static = None, transforms_x_dynamic = None, transforms_y_static = None, transforms_y_dynamic = None):
    self.dataset = []
    self.transforms_x = transforms_x_dynamic
    self.transforms_y = transforms_y_dynamic
    for category, img_files in getImagesPaths(root_path):
        # Only the green lens image
        img = pil.Image.open(img_files[-1]) 
        if transforms_x_static != None:
          img = transforms_x_static(img)
        if transforms_y_static != None:
          category = transforms_y_static(category)
        self.dataset.append((img, category))    

class TransformsRand:
  def __init__(self):
    self.angle = random.random()
    self.scale = random.random()
    self.shear = random.random()
    self.hflip = random.random()
        
class AllImagesDataset(t.utils.data.Dataset):
  def __init__(self, root_path, transforms_x_static = None, transforms_x_dynamic = None, transforms_y_static = None, transforms_y_dynamic = None, is_train = True):
    self.is_train = is_train
    self.dataset = []
    self.transforms_x = transforms_x_dynamic
    self.transforms_y = transforms_y_dynamic
    for category, img_files in getImagesPaths(root_path):
      imgs = []
      for i in range(7):
        img = pil.Image.open(img_files[i])
        if transforms_x_static != None:
          img = transforms_x_static(img)
        imgs.append(img)
      if transforms_y_static != None:
        category = transforms_y_static(category)
      self.dataset.append((imgs, category))    
  
  def __getitem__(self, i):
    x, y = self.dataset[i]
    if self.transforms_x != None:
      if self.is_train:
        rand = TransformsRand()
        x = [self.transforms_x(_x, rand = rand) for _x in x]
#         x = [self.transforms_x(_x) for _x in x]
      else:
        x = [self.transforms_x(_x) for _x in x]
    if self.transforms_y != None:
      y = self.transforms_y(y)
    return x, y

  def __len__(self):
    return len(self.dataset)

**Preprocess Data**

Convert pytorch tensor to numpy array.

In [0]:
def to_numpy(x):
  return x.cpu().detach().numpy()

Data transformations for the test and training sets.

In [0]:
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

def custom_transforms(x, angle = 45, scale = (1., 2.), shear = 30, rand = None):
    if rand == None:
        rand = TransformsRand()
    angle = angle * rand.angle
    scale_value = scale[0] + ((scale[1] - scale[0]) * rand.scale)
    shear = shear * rand.shear
    x = tv.transforms.functional.affine(x, angle = angle, scale = scale_value, shear = shear, translate = [0, 0])
    x = tv.transforms.functional.resize(x, IMAGE_SIZE)
    if rand.hflip > .5:
        x = tv.transforms.functional.hflip(x)
    x = tv.transforms.functional.to_tensor(x).cuda()
    x = tv.transforms.functional.normalize(x, mean=norm_mean, std=norm_std)
    return x

transforms_train = tv.transforms.Compose([
    tv.transforms.RandomAffine(degrees  = 45, translate = None, scale = (1., 2.), shear = 30),
    # tv.transforms.CenterCrop(CROP_SIZE),
    tv.transforms.Resize(IMAGE_SIZE),
    tv.transforms.RandomHorizontalFlip(),
    tv.transforms.ToTensor(),
    tv.transforms.Lambda(lambda t: t.cuda()),
    tv.transforms.Normalize(mean=norm_mean, std=norm_std)    
])

transforms_test = tv.transforms.Compose([
    # tv.transforms.CenterCrop(CROP_SIZE),
    tv.transforms.Resize(IMAGE_SIZE),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize(mean=norm_mean, std=norm_std)    
])

y_transform = tv.transforms.Lambda(lambda y: t.tensor(y, dtype=t.long, device = 'cuda:0'))

Initialize pytorch datasets and loaders for training and test.

In [0]:
def create_loaders():
  dataset_train = AllImagesDataset(TRAIN_PATH, transforms_x_dynamic = custom_transforms, transforms_y_dynamic = y_transform)
  dataset_test = AllImagesDataset(TEST_PATH, transforms_x_static = transforms_test, 
        transforms_x_dynamic = tv.transforms.Lambda(lambda t: t.cuda()), transforms_y_dynamic = y_transform, is_train = False)

  loader_train = DataLoader(dataset_train, BATCH_SIZE, shuffle = True, num_workers = 0)
  loader_test = DataLoader(dataset_test, BATCH_SIZE, shuffle = False, num_workers = 0)
  return loader_train, loader_test, len(dataset_train), len(dataset_test)

In [0]:
loader_train_simple_img, loader_test_simple_img, len_train, len_test = create_loaders()

**Visualize Data**

Load a few images so that we can see the effects of the data augmentation on the training set.

In [0]:
def plot_one_prediction(x, label, pred):  
  x, label, pred = to_numpy(x), to_numpy(label), to_numpy(pred)
  x = np.transpose(x, [1, 2, 0])
  if x.shape[-1] == 1:
    x = x.squeeze()
  x = x * np.array(norm_std) + np.array(norm_mean)
  plt.title(label, color = 'green' if label == pred else 'red')
  plt.imshow(x)

def plot_predictions(imgs, labels, preds):  
  fig = plt.figure(figsize = (20, 5))
  for i in range(20):
    fig.add_subplot(2, 10, i + 1, xticks = [], yticks = [])
    plot_one_prediction(imgs[i], labels[i], preds[i])

In [0]:
# x, y = next(iter(loader_train_simple_img))
# for i in range(7):
#   plot_predictions(x[i], y, y)

**Model**

Define a few models to experiment with.

In [0]:
def get_mobilenet_v2():
  model = t.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=True)
  model.classifier[0] = t.nn.Dropout(p=0.9, inplace=False)
  model.classifier[1] = Linear(in_features=1280, out_features=4, bias=True)
  model.features[18].add_module('cnn_drop_18', t.nn.Dropout2d(p = .3))
  model.features[17]._modules['conv'][1].add_module('cnn_drop_17', t.nn.Dropout2d(p = .2))
  model.features[16]._modules['conv'][1].add_module('cnn_drop_16', t.nn.Dropout2d(p = .1))
  model = model.cuda()
  return model

def get_vgg_19():
  model = tv.models.vgg19(pretrained = True)
  model = model.cuda()
  model.classifier[2].p = .9
  model.classifier[6].out_features = 4
  return model

def get_res_next_101():
  model = t.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl')
  model.fc = t.nn.Sequential(
      t.nn.Dropout(p = .9),
      t.nn.Linear(in_features=2048, out_features=4)
  )
  model = model.cuda()
  return model

def get_resnet_18():
  model = tv.models.resnet18(pretrained = True)
  model.fc = t.nn.Sequential(
      t.nn.Dropout(p = .9),
      t.nn.Linear(in_features=512, out_features=4)
  )
  model = model.cuda()
  return model

def get_dense_net():
  model = tv.models.densenet121(pretrained = True)
  model.classifier = t.nn.Sequential(
      t.nn.Dropout(p = .9),
      t.nn.Linear(in_features = 1024, out_features = 4)
  )
  model = model.cuda()
  return model

Define ensemble.

In [0]:
class WrappedModel(t.nn.Module):
	def __init__(self, module):
		super().__init__()
		self.module = module # that I actually define.
	def forward(self, x):
		return self.module(x)

class MobileNetV2_FullConv(t.nn.Module):
  def __init__(self, end_channels):
    super().__init__()
    self.cnn = get_mobilenet_v2().features
    self.cnn[18] = t.nn.Sequential(
      tv.models.mobilenet.ConvBNReLU(320, end_channels, kernel_size=1)
    )
    self.fc = t.nn.Linear(end_channels, 4)

  def forward(self, x):
    x = self.cnn(x)
    x = x.mean([2, 3])
    x = self.fc(x);
    return x
    
class Ensamble(t.nn.Module):
  def __init__(self):
    super().__init__()
    self.model_simple = cnn_full_conv(CHECKPOINT_NATURAL_IMG_MODEL, 32).cnn
    self.model_green = cnn_full_conv(CHECKPOINT_GREEN_LENS_IMG_MODEL, 64).cnn
    self.model_red = cnn_full_conv(CHECKPOINT_IODINE_SOLUTION_IMG_MODEL, 32).cnn
    channels = 32 * 5 + 64 + 32
    fc_size = 32
    self.classifier = tv.models.mobilenet.InvertedResidual(inp = channels, oup = fc_size, stride = 1, expand_ratio = 5)
    self.classifier._modules['conv'][0].add_module('classifier_drop_1', t.nn.Dropout2d(p = .6))
    self.classifier._modules['conv'][1].add_module('classifier_drop_1', t.nn.Dropout2d(p = .6))
    self.fc = t.nn.Sequential(
        # t.nn.Dropout(p = .2),
        t.nn.Linear(fc_size, 4)
    )

  def forward(self, x):
    x_list = []
    for i in range(5):
      x_list.append(self.model_simple(x[i]))
    x_list.append(self.model_green(x[5]))
    x_list.append(self.model_red(x[6]))
    x_concat = t.cat(x_list, 1)
    x_concat = self.classifier(x_concat)
    x_concat = x_concat.mean([2, 3])
    x_concat = self.fc(x_concat)
    return x_concat

def cnn(checkpoint_path):
  cnn = t.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=False)
  cnn.classifier[0] = t.nn.Dropout(p=0, inplace=False)
  cnn.classifier[1] = Linear(in_features=1280, out_features=4, bias=True)
  checkpoint = t.load(checkpoint_path)
  cnn.load_state_dict(checkpoint['model'])
  for param in cnn.parameters():
    param.requires_grad = False
  return cnn

def cnn_from_data_parallel(checkpoint_path):
  cnn = t.hub.load('pytorch/vision', 'mobilenet_v2', pretrained=False)
  cnn.classifier[0] = t.nn.Dropout(p=0, inplace=False)
  cnn.classifier[1] = Linear(in_features=1280, out_features=4, bias=True)
  cnn = WrappedModel(cnn)
  checkpoint = t.load(checkpoint_path)
  cnn.load_state_dict(checkpoint['model'])
  for param in cnn.parameters():
    param.requires_grad = False
  return cnn

def cnn_full_conv(checkpoint_path, end_channels_nb):  
  cnn = MobileNetV2_FullConv(end_channels_nb)
  checkpoint = t.load(checkpoint_path)
  cnn.load_state_dict(checkpoint['model'])
  for param in cnn.parameters():
    param.requires_grad = False
  return cnn

**Evaluate**

In [0]:
model_4_class = t.nn.DataParallel(Ensamble().cuda())
checkpoint_4_class = t.load(CHECKPOINT_ENSAMBLE)
model_4_class.load_state_dict(checkpoint_4_class['model'])

Using cache found in /root/.cache/torch/hub/pytorch_vision_master
Using cache found in /root/.cache/torch/hub/pytorch_vision_master
Using cache found in /root/.cache/torch/hub/pytorch_vision_master


<All keys matched successfully>

In [0]:
def evaluate(model, loader_test, len_test):
  test_acc, test_precision, test_recall, test_f_score = 0, 0, 0, 0
  model.eval()
  with t.no_grad():
    for x, y in loader_test:
      y_pred = model.forward(x)
      y_pred, y = to_numpy(y_pred), to_numpy(y)
      pred = y_pred.argmax(axis = 1)
      pred = pred > 1
      y = y > 1
      ratio = len(y) / len_test
      test_acc += (sk.metrics.accuracy_score(y, pred) * ratio)
      precision, recall, f_score, _ = sk.metrics.precision_recall_fscore_support(y, pred, average = 'macro')
      test_precision += (precision * ratio)
      test_recall += (recall * ratio)
      test_f_score += (f_score * ratio)
  print('Acc {} prec {} rec {} f {}'.format(test_acc, test_precision, test_recall, test_f_score))
  return test_acc, test_precision, test_recall, test_f_score

In [0]:
evaluate(model_4_class, loader_test_simple_img, len_test)

Acc 0.8958333333333334 prec 0.8965217391304348 rec 0.8958333333333333 f 0.8957881024750325


(0.8958333333333334,
 0.8965217391304348,
 0.8958333333333333,
 0.8957881024750325)